In [15]:
import torch
from numpy import mean
from math import sqrt
from fairlearn.datasets import fetch_adult
from sklearn.model_selection import train_test_split
from fairlearn.adversarial import AdversarialFairnessClassifier
from fairlearn.metrics import equalized_odds_ratio, demographic_parity_difference
import pickle


In [3]:
from sklearn.compose import make_column_transformer, make_column_selector
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from numpy import number

ct = make_column_transformer(
    (
        Pipeline(
            [
                ("imputer", SimpleImputer(strategy="mean")),
                ("normalizer", StandardScaler()),
            ]
        ),
        make_column_selector(dtype_include=number),
    ),
    (
        Pipeline(
            [
                ("imputer", SimpleImputer(strategy="most_frequent")),
                ("encoder", OneHotEncoder(drop="if_binary", sparse=False)),
            ]
        ),
        make_column_selector(dtype_include="category"),
    ),
)

In [4]:


X, y = fetch_adult(return_X_y=True)
pos_label = y[0]

z = X["sex"] # In this example, we consider 'sex' the sensitive feature.


X_train, X_test, Y_train, Y_test, Z_train, Z_test = train_test_split(
    X, y, z, test_size=0.2, random_state=12345, stratify=y
)

X_prep_train = ct.fit_transform(X_train) # Only fit on training data!
X_prep_test = ct.transform(X_test)

/Users/uni/miniconda3/envs/ma/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [5]:
class PredictorModel(torch.nn.Module):
    def __init__(self):
        super(PredictorModel, self).__init__()
        self.layers = torch.nn.Sequential(
            torch.nn.Linear(X_prep_train.shape[1], 200),
            torch.nn.LeakyReLU(),
            torch.nn.Linear(200, 1),
            torch.nn.Sigmoid(),
        )

    def forward(self, x):
        return self.layers(x)



predictor_model = PredictorModel()

In [6]:
def validate(mitigator):
    predictions = mitigator.predict(X_prep_test)
    dp_diff = demographic_parity_difference(
        Y_test == pos_label,
        predictions == pos_label,
        sensitive_features=Z_test,
    )
    accuracy = mean(predictions.values == Y_test.values)
    selection_rate = mean(predictions == pos_label)
    print(
        "DP diff: {:.4f}, accuracy: {:.4f}, selection_rate: {:.4f}".format(
            dp_diff, accuracy, selection_rate
        )
    )
    return dp_diff, accuracy, selection_rate



In [7]:
schedulers = []

def optimizer_constructor(model):
    global schedulers
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    schedulers.append(
        torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.995)
    )
    return optimizer

step = 1


In [8]:
def callbacks(model, *args):
    global step
    global schedulers
    step += 1
    # Update hyperparameters
    model.alpha = 0.3 * sqrt(step // 1)
    for scheduler in schedulers:
        scheduler.step()
    # Validate (and early stopping) every 50 steps
    if step % 50 == 0:
        dp_diff, accuracy, selection_rate = validate(model)
        # Early stopping condition:
        # Good accuracy + low dp_diff + no mode collapse
        if (
            dp_diff < 0.03
            and accuracy > 0.8
            and selection_rate > 0.01
            and selection_rate < 0.99
        ):
            return True


In [11]:
mitigator = AdversarialFairnessClassifier(
    predictor_model=predictor_model,
    adversary_model=[3, "leaky_relu"],
    predictor_optimizer=optimizer_constructor,
    adversary_optimizer=optimizer_constructor,
    epochs=10,
    batch_size=2 ** 7,
    shuffle=True,
    callbacks=callbacks,
    random_state=123,
)

In [78]:
# X_prep_test = X_prep_test.double()
# AdversarialFairnessClassifier.threshold
mitigator.threshold_value
predictor_model.forward(torch.tensor(X_prep_test).float()) > 0.5


# import numpy as np
# np.array(X_test)
# # X_test
# X_prep_train
# torch.tensor(X_prep_test).float().dtype
# X_prep_train.shape[1]


tensor([[False],
        [False],
        [ True],
        ...,
        [False],
        [False],
        [False]])

In [14]:
mitigator.fit(X_prep_train, Y_train, sensitive_features=Z_train)

DP diff: 0.0079, accuracy: 0.7668, selection_rate: 0.9934
DP diff: 0.7883, accuracy: 0.6236, selection_rate: 0.4692
DP diff: 0.0833, accuracy: 0.8031, selection_rate: 0.9353
DP diff: 0.1183, accuracy: 0.8355, selection_rate: 0.7984
DP diff: 0.2121, accuracy: 0.7642, selection_rate: 0.8546
DP diff: 0.2238, accuracy: 0.7664, selection_rate: 0.8024
DP diff: 0.2027, accuracy: 0.7742, selection_rate: 0.7973
DP diff: 0.1497, accuracy: 0.7868, selection_rate: 0.8695
DP diff: 0.0863, accuracy: 0.8066, selection_rate: 0.8527
DP diff: 0.1119, accuracy: 0.8021, selection_rate: 0.8046
DP diff: 0.1195, accuracy: 0.7979, selection_rate: 0.8548
DP diff: 0.1345, accuracy: 0.7961, selection_rate: 0.8262
DP diff: 0.1142, accuracy: 0.8023, selection_rate: 0.8433
DP diff: 0.1048, accuracy: 0.8051, selection_rate: 0.8342
DP diff: 0.0895, accuracy: 0.8072, selection_rate: 0.8085
DP diff: 0.0841, accuracy: 0.8096, selection_rate: 0.8231
DP diff: 0.0906, accuracy: 0.8083, selection_rate: 0.8185
DP diff: 0.095

AdversarialFairnessClassifier(adversary_model=[3, 'leaky_relu'],
                              adversary_optimizer=<function optimizer_constructor at 0x169be4a60>,
                              alpha=16.730212192318422, batch_size=128,
                              callbacks=<function callbacks at 0x169be4af0>,
                              epochs=10,
                              predictor_model=PredictorModel(
  (layers): Sequential(
    (0): Linear(in_features=104, out_features=200, bias=True)
    (1): LeakyReLU(negative_slope=0.01)
    (2): Linear(in_features=200, out_features=1, bias=True)
    (3): Sigmoid()
  )
),
                              predictor_optimizer=<function optimizer_constructor at 0x169be4a60>,
                              random_state=123, shuffle=True)

In [20]:
predictor_model.forward(X_test)

TypeError: linear(): argument 'input' (position 1) must be Tensor, not DataFrame

In [16]:
    with open("test.pkl", "wb") as f:
        pickle.dump(mitigator, f)

AttributeError: Can't pickle local object '_AdversarialFairness._set_predictor_function.<locals>.<lambda>'

Save state dicts and somehow implement a predictir function...
just pass to forward..